# <center>[⏱ Оптимизация памяти и ускорение вычислений](https://stepik.org/lesson/825508/)</center>

### Оглавление ноутбука

<img src='../images/speed_up.png' align="right" width="550" height="550" />
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c2">Cчитывание и сохранение больших датафреймов</a></li>
<li><a href="#c2">Оптимизация памяти </a></li>
<li><a href="#c2">Ускорение при помощи numpy</a></li>
<li><a href="#c2">Векторизация в pandas</a></li>
<li><a href="#c2">Numba.jit</a></li>
<li><a href="#c2">Мультипроцессинг</a></li>
<li><a href="#6">Выводы и заключения</a>

</li></ul></font></p>

    

<div class="alert alert-info">
Когда речь заходит о работе с действительно большими данными скорость работы программы и кол-во памяти, которое ей требуется могут стать одним из главных боттлнеков в вашей программе. Особенно если вы ограничены в ресурсах (как например часто бывает на Kaggle) и вам нужно, чтобы ваше решение отработало не только четко, но еще и быстро. В этом уроке мы рассмотрим подходы и методы с помощью которых можно сэкономить память и ускорить ваши вычисления.

## Импортируем библиотеки

In [3]:
import pandas as pd
import numpy as np

# <center> 👁 Считывание и сохранение больших датафреймов </center>

### Работа с pickle

<div class="alert alert-info">

`Pickle` это отличная альтернатива привычным нам `.csv` файлам при работе с большими файлами. Мало того, что он считывает и сохраняет все в разы быстрее, так еще и место на диске такой файл занимает меньше. Также при использование `to_pickle()` сохраняются индексы и все типы колонок, так что при его последующем считывании датафрейм будет точно таким же, и его не нужно будет повторно оптимизировать при каждом открытии, как при использовании CSV формата.

In [9]:
%%time

data = pd.read_csv('../data/blending/text_classification_train.csv')

CPU times: user 1.6 s, sys: 307 ms, total: 1.91 s
Wall time: 1.98 s


In [11]:
data.shape

(7500, 2618)

In [12]:
%%time

data.to_csv('../data/blending/text_classification_train.csv')

CPU times: user 8.63 s, sys: 449 ms, total: 9.08 s
Wall time: 9.14 s


In [13]:
%%time

pd.to_pickle(data, '../data/blending/text_classification_train.pickle')

CPU times: user 15.6 ms, sys: 27.3 ms, total: 42.9 ms
Wall time: 54.7 ms


In [235]:
%%time

data = pd.read_pickle('../data/blending/text_classification_train.pickle')

CPU times: user 11.8 ms, sys: 39.4 ms, total: 51.1 ms
Wall time: 82.1 ms


In [237]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Columns: 2618 entries, category to labse_text_feature_767
dtypes: float64(2616), object(2)
memory usage: 149.8+ MB


### Считывание по батчам

<div class="alert alert-info">

Если ваш датасет не умещается в память без оптимизации типов или он не нужен вам целиком, то можно считывать по батчам и сразу указывать необхоимые типы, индексы и тд. В параметр `chunksize` - передается число сэмплов, которое будет считываться за 1 итерацию.

In [40]:
import gc

chunksize = 1000
tmp_lst = []
with pd.read_csv('../data/car_train.csv',
                 index_col='car_id',
                 dtype={'model': 'category',
                        'car_type': 'category',
                        'fuel_type': 'category',
                        'target_class': 'category'}, chunksize=chunksize) as reader:
    for chunk in reader:
        tmp_lst.append(chunk)
        
data = pd.concat(tmp_lst)

del tmp_lst
gc.collect()

data.head()

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
car_id,,,,,,,,,
y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug
O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick
u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel
N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel


***Важно:*** Пока на какой-то объект в памяти есть ссылки, которые явно не удалили или не переназначили - этот объект будет занимать оперативную память, хотя он может больше не использоваться. Поэтому все временные объекты, которые больше не будут использоваться, лучше явно удалять, используя `del` и после этого запускать сборщик мусора gc - `garbage collector`,как в примере выше.
```python
import gc
gc.collect()
```

### Используем генератор

<div class="alert alert-info">
Это особенно актуально, когда мы работаем с картинками - чтобы не хранить их всех в оперативной памяти, они считываются только перед тем как модель хочет посчитать по ним ошибку и скорректировать веса. Однако при работе с большими текстами тоже бывает полезно.

In [344]:
def read_file(filename):
    with open(filename, 'r') as f:
        for line in f:
            yield line.strip()

In [350]:
it = read_file('../data/car_info.csv')
next(it)

'car_type,fuel_type,car_rating,year_to_start,riders,car_id,model,target_class,year_to_work,target_reg'

In [351]:
next(it)

'economy,petrol,4.8,2013,42269,P17494612l,Skoda Rapid,engine_overheat,2019,46.65071890960271'

# <center> 🗜 Оптимизация памяти </center>

<div class="alert alert-info">

Самый эффективный способ оптимизации памяти, если вы не хотите удалять часть данных, это установка правильных типов. Если колонка имеет тип `int`, то не нужно ставить ей тип `float`, а если в ней всего несколько уникальных значений, то не нужно делать ее типом `string`. .Например, если в нашем датасете есть бинарная колонка в которой хранятся только 0 и 1, `pandas` хранит её как максимально возможный тип для целых чисел `int64`, хотя достаточно будет `int8`. При правильной постановке типов размер нового датасета обычно в несколько раз меньше (а то и на порядок), чем без них. Так же можно вынести какую-то колонку как индекс, чтобы не хранить лишний индекс, но это уже косметика.

### Настройка индексов

In [18]:
data = pd.read_csv('../data/blending/text_classification_train.csv')
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Columns: 2619 entries, Unnamed: 0 to labse_text_feature_767
dtypes: float64(2616), int64(1), object(2)
memory usage: 156.4 MB


In [19]:
data = pd.read_csv('../data/blending/text_classification_train.csv', index_col='Unnamed: 0')
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7500 entries, 0 to 7499
Columns: 2618 entries, category to labse_text_feature_767
dtypes: float64(2616), object(2)
memory usage: 156.4 MB


### Оптимизируем числовые типы

In [20]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        if col_type not in ['object', 'category', 'datetime64[ns, UTC]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [21]:
data = reduce_mem_usage(data)
data.info()

Memory usage of dataframe is 149.86 MB
Memory usage after optimization is: 37.59 MB
Decreased by 74.9%
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7500 entries, 0 to 7499
Columns: 2618 entries, category to labse_text_feature_767
dtypes: float16(2616), object(2)
memory usage: 37.6+ MB


### Оптимизируем категориальные фичи

In [307]:
df_cars = pd.read_csv('../data/car_train.csv')
df_cars.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel


In [261]:
df_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   car_id         2337 non-null   object 
 1   model          2337 non-null   object 
 2   car_type       2337 non-null   object 
 3   fuel_type      2337 non-null   object 
 4   car_rating     2337 non-null   float64
 5   year_to_start  2337 non-null   int64  
 6   riders         2337 non-null   int64  
 7   year_to_work   2337 non-null   int64  
 8   target_reg     2337 non-null   float64
 9   target_class   2337 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 182.7+ KB


In [262]:
def convert_columns_to_catg(df, column_list):
    for col in column_list:
        print("converting", col.ljust(30), "size: ", round(df[col].memory_usage(deep=True)*1e-6,2), end="\t")
        df[col] = df[col].astype("category")
        print("->\t", round(df[col].memory_usage(deep=True)*1e-6,2))

In [263]:
convert_columns_to_catg(df_cars, ['model', 'car_type', 'fuel_type', 'target_class'])

converting model                          size:  0.16	->	 0.01
converting car_type                       size:  0.15	->	 0.0
converting fuel_type                      size:  0.15	->	 0.0
converting target_class                   size:  0.16	->	 0.0


In [264]:
df_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   car_id         2337 non-null   object  
 1   model          2337 non-null   category
 2   car_type       2337 non-null   category
 3   fuel_type      2337 non-null   category
 4   car_rating     2337 non-null   float64 
 5   year_to_start  2337 non-null   int64   
 6   riders         2337 non-null   int64   
 7   year_to_work   2337 non-null   int64   
 8   target_reg     2337 non-null   float64 
 9   target_class   2337 non-null   category
dtypes: category(4), float64(2), int64(3), object(1)
memory usage: 120.7+ KB


In [257]:
df_cars.memory_usage().sum() / 1024

120.734375

In [258]:
convert_columns_to_catg(df_cars, ['car_id'])

converting car_id                         size:  0.16	->	 0.23


In [259]:
df_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   car_id         2337 non-null   category
 1   model          2337 non-null   category
 2   car_type       2337 non-null   category
 3   fuel_type      2337 non-null   category
 4   car_rating     2337 non-null   float64 
 5   year_to_start  2337 non-null   int64   
 6   riders         2337 non-null   int64   
 7   year_to_work   2337 non-null   int64   
 8   target_reg     2337 non-null   float64 
 9   target_class   2337 non-null   category
dtypes: category(5), float64(2), int64(3)
memory usage: 189.8 KB


In [265]:
df_cars = reduce_mem_usage(df_cars)
df_cars.memory_usage().sum() / 1024

Memory usage of dataframe is 0.12 MB
Memory usage after optimization is: 0.06 MB
Decreased by 52.9%


56.83203125

In [267]:
df_cars.to_pickle('../small_df_cars.pickle')

In [287]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/779914/make_gold_features.csv')
df.head()

,car_id,ride_date,deviation_normal,target_class
0,b12101843B,2020-01-01,-2.846,engine_ignition
1,b12101843B,2020-01-02,-2.431,engine_ignition
2,b12101843B,2020-01-03,-5.992,engine_ignition
3,b12101843B,2020-01-04,-3.427,engine_ignition
4,b12101843B,2020-01-05,-6.351,engine_ignition


In [288]:
df['dif'] = df.groupby('car_id')['deviation_normal'].diff()
df.head()

,car_id,ride_date,deviation_normal,target_class,dif
0,b12101843B,2020-01-01,-2.846,engine_ignition,NaN
1,b12101843B,2020-01-02,-2.431,engine_ignition,0.415
2,b12101843B,2020-01-03,-5.992,engine_ignition,-3.561
3,b12101843B,2020-01-04,-3.427,engine_ignition,2.565
4,b12101843B,2020-01-05,-6.351,engine_ignition,-2.924


In [289]:
res2 = df.groupby('car_id')['dif'].agg(['max', np.argmax]).reset_index().rename(columns={'max': 'gold_feature_1',
                                                                                  'argmax': 'gold_feature_2'})

In [290]:
res1.head()

,car_id,gold_feature_1,gold_feature_2
0,B-8479157Q,7.773,58
1,C14604474J,20.931,50
2,D46039134C,18.446,48
3,E-1571710h,26.922,53
4,F51416137Z,43.798,49


In [292]:
(res2.values == res1.values).all()

True

## <center>🥌 Ускорение при помощи `Numpy`</center>

<div class="alert alert-info">

Первое правило быстрого кода - используйте `numpy` везде, где можно. Он исполняется на чисто C, так что в работает в сотни раз быстрее обычных циклов и list-ов. В общем, если можете написать код при помощи `numpy` - пишите.

### Инициализация

<div class="alert alert-info">

Демонстрация скорости работы `numpy` на примере инициализации.

In [52]:
%%timeit

a = list(range(1_000_000))

9.39 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [57]:
%%timeit

b = np.arange(1_000_000)

654 µs ± 29 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [79]:
a = list(range(1_000_000))

In [80]:
b = np.arange(1_000_000)

In [81]:
%%timeit
100000 in a

475 µs ± 3.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [82]:
%%timeit
100000 in b

360 µs ± 2.72 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Поэлементные функции

<div class="alert alert-info">

Демонстрация скорости работы `numpy` на примере поэлементных функций.

In [53]:
%%timeit
[el * el for el in a]

47.4 ms ± 390 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [56]:
%%timeit
[el + 10 for el in a]

53.4 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [55]:
%%timeit
b * b

670 µs ± 18.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [60]:
%%timeit
b + 10

649 µs ± 24.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Aггрегирующие функции

<div class="alert alert-info">

Методы и функции `numpy` работают быстрее, чем их обычные аналоги в питоне.

In [62]:
%%timeit

max(b)

25.9 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [63]:
%%timeit

b.max()

100 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Когда использовать List?

<div class="alert alert-info">

На самом деле в одном случае `list` все-таки может быть быстрее. Если вы хотите добавлять много новых элементов, то добавление одного элемента в `list` происходит за O(1) и суммарно на все тратится O(n) времени, а при добавление в `numpy` массив нам нужно его заново весь проинициализировать, даже если добавили всего один элемент, так что суммарное время работы будет O(n^2).

In [67]:
%%timeit

a = np.zeros(0)
for el in range(1000):
    b = np.zeros(1000)
    a = np.append(a, b)

302 ms ± 28.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
%%timeit

a = []
for el in range(1000):
    b = [0 for i in range(1000)]
    a += b

20.2 ms ± 373 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Используем `set()`

<div class="alert alert-info">

Если нужно проверить наличие элементов в другом массиве и тот "другой" достаточно большой, то лучше использовать `set()`, так в нем поиск элемента осуществляется за O(log(n)), а в `np.isin()` за O(n). Так что даже несмотря на то, что `numpy` оптимизрован `set()` выигрывает его по времени.

In [87]:
a = np.arange(1000)
b = np.arange(1000000) * -1

In [88]:
%%timeit
np.isin(a, b)

21.1 ms ± 530 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [89]:
st = set(b)

In [90]:
%%timeit
[el in st for el in a]

41 µs ± 243 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## <center> 🍢 Векторизация в `pandas`. </center>

<div class="alert alert-info">
Векторизация - это процесс, когда мы заменяем обычные поэлементные функции на операции сразу со всем вектором значений, что позволяет значительно прибавить в скорости.

In [96]:
df_cars.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel


In [295]:
df_cars['car_type'].unique()

['economy', 'standart', 'business', 'premium']
Categories (4, object): ['business', 'economy', 'premium', 'standart']

### Numpy.where()

<div class="alert alert-info">

Позволяет проверить какое-то условие и в зависимости от его результатов вернуть то или иное значение. Векторный аналог `if-else`.

In [110]:
def simple_if(x):
    if x['car_rating'] < 3.78:
        return x['car_type']
    else:
        return x['fuel_type']

In [112]:
%%timeit
df_cars.apply(simple_if, axis=1)

7.44 ms ± 59.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [113]:
%%timeit
np.where(df_cars['car_rating'].values < 3.78, df_cars['car_type'].values, df_cars['fuel_type'].values)

19 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Numpy.vectorize()

<div class="alert alert-info">
Часть функции можно оставить поэлементными и просто засчет их векторизации получить значительный прирост скорости. Однако работает это все равно медленне, чем операции с векторами чисел.

In [114]:
def simple_if2(car_rating, car_type, fuel_type):
    if car_rating < 3.78:
        return car_type
    else:
        return fuel_type

In [117]:
vectfunc = np.vectorize(simple_if2)

In [120]:
%%timeit
vectfunc(df_cars['car_rating'], df_cars['car_type'], df_cars['fuel_type'])

349 µs ± 5.72 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Numpy.select()

<div class="alert alert-info">

Как `np.where()`, только для нескольких условий.

In [138]:
import re

def hard_if(x):
    if x['car_rating'] < 3:
        if 'Audi' == x['model']:
            return 0
        else:
            return 1
    elif x['car_rating'] in [3, 4, 5]:
        return 2
    else:
        return 3

In [139]:
%%timeit
df_cars.apply(simple_if, axis=1)

7.38 ms ± 53.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [140]:
%%timeit
conditions = [
    (df_cars['car_rating'] < 3) & (df_cars['model'] == 'Audi'),
    (df_cars['car_rating'] < 3),
    df_cars['car_rating'].isin([3, 4, 5])
]

choices = [0, 1, 2]
np.select(conditions, choices, default=3)

347 µs ± 4.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Переписываем словари

<div class="alert alert-info">
На самом деле, обращаться к значению из словаря тоже можно быстро.

In [147]:
mydict = {'economy': 0,
          'standart': 1,
          'business': 2,
          'premium': 3}
def f(x):
    if x['car_rating'] > 5:
        return mydict[x['car_type']]
    else:
        return np.nan

In [148]:
%%timeit
df_cars.apply(simple_if, axis=1)

7.65 ms ± 75.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [151]:
%%timeit
np.where(df_cars['car_rating'] > 5, df_cars['car_type'].map(mydict), np.nan)

222 µs ± 4.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Пишем groupby на numpy

<div class="alert alert-info">

`pd.groupby()` - одна из самых часто используемых и полезных функций в пандасе, которую можно ускорить до 30 раз при помощи `numpy`!

In [339]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
df_cars['int_model'] = lbl.fit_transform((df_cars['model'] + df_cars['fuel_type']).astype(str))

In [173]:
%%timeit
df_cars.groupby(['model', 'fuel_type'])['target_reg'].sum()

340 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [174]:
%%timeit
np.bincount(df_cars['int_model'], weights=df_cars['target_reg'])

9.63 µs ± 35.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [333]:
%%timeit
indices = df_cars['int_model']
max_values = np.maximum.reduceat(df_cars['target_reg'].values[np.argsort(indices)],
                                 np.concatenate(([0], np.cumsum(np.bincount(indices))))[:-1])
max_values

54.9 µs ± 2.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [334]:
%%timeit
df_cars.groupby(['int_model'])['target_reg'].agg(['max'])

183 µs ± 777 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## <center> `Numba Jit` </center>

<div class="alert alert-info">

Если нужно что-то кастномное, что нельзя переписать на `numpy`, но нужно чтобы работало быстро, `numba.jit` вам в помощь. Он конвертирует написанный вами на питоне код в С и засчет этого работает на порядок (а иногда и на несколько) быстрее, чем обычный питон. Однако, поддерживает он не весь питоновский функционал и внутри него нельзя использовать разные библиотечные функции, так что в основном используется для низкоуровневой оптимизации.

In [357]:
from numba import jit

@jit(nopython=True)
def f(n):
    s = 0.
    for i in range(n):
        s += i ** 0.5
    return s

In [356]:
%%timeit
f(10000)

535 µs ± 2.48 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [358]:
%%timeit
f(10000)

11.8 µs ± 2.17 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [400]:
@jit(nopython=True)
def monotonically_increasing(a):
    max_value = 0
    for i in range(len(a)):
        if a[i] > max_value:
            max_value = a[i]
        a[i] = max_value
    return a

In [399]:
%%timeit
monotonically_increasing(df_cars['target_reg'].values)

205 µs ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [401]:
%%timeit
monotonically_increasing(df_cars['target_reg'].values)

4.49 µs ± 9.07 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Многопоточное apply

### Swifter

<div class="alert alert-info">
Позволяет распараллелить apply на несколько потоков (полезно, когда io bounds задача, а не cpu-bounds).

In [179]:
!pip install swifter -q

In [181]:
import swifter

In [230]:
import sys
sys.path.append('../src/')
import text_prepare
import importlib
importlib.reload(text_prepare)

<module 'text_prepare' from '/Users/sergak/Documents/Neyro_sets/Competitive_Data_Science/notebooks/../src/text_prepare.py'>

In [221]:
df = pd.read_csv('../data/blending/text_transformer_data.csv')
df.head()

,category,text
0,extreme,Ледник Пасторури это цирковой ледник расположе...
1,martial_arts,Главные участники предстоящего Betokenoid 274 ...
2,extreme,Ttokenoid Btokenoid – карта с которой можно не...
3,autosport,В Сильверстоуне произошли крупные обновления а...
4,extreme,На протяжении более чем 30 лет Вестсайд являет...


In [232]:
%%time

tmp = df['text'].apply(text_prepare.text_prepare)

CPU times: user 17.5 s, sys: 3.87 s, total: 21.3 s
Wall time: 21.6 s


In [233]:
%%time

tmp = df['text'].swifter.apply(text_prepare.text_prepare)

Pandas Apply:   0%|          | 0/500 [00:00<?, ?it/s]

CPU times: user 19.1 s, sys: 4.24 s, total: 23.3 s
Wall time: 23.4 s


## <center> 🧵 Multiprocessing </center>

<div class="alert alert-info">
 
С помощью `multiprocessing` можно ускорить вообще практически все. Он позвлоляет использовать не одно ядро вашего компьютера, а сразу несколько и соответственно ускорить вычисления (уже не только io-bound, но еще и cpu-bound) в кол-во раз пропорциональное кол-ву доступных ядер.

In [220]:
from multiprocessing import Pool

In [228]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = np.concatenate(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [231]:
%%time
tmp = parallelize_dataframe(df, text_prepare.many_row_prepare)

CPU times: user 10.2 ms, sys: 39 ms, total: 49.2 ms
Wall time: 6.41 s


## Выводы

<div class="alert alert-info">

Ускорение вычеслений и оптимизация памяти это важные задачи, с которыми периодически можно сталкнутся во время написания соревнований или просто при работе с большим кол-вом данных. В данном уроке мы рассмотрели основные способы решения этих задач и как их применять на практике. Не тормози, съешь `numpy`!